In [1]:
!pip install tflearn

import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import tensorflow as tf
import numpy as np
import tflearn
import random
import json

     |████████████████████████████████| 112kB 2.9MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-cp37-none-any.whl size=127301 sha256=d6d8bdfbdaef15b667a75330123bec7508d15267bbcba139fb91dd6fb03cc2f2
  Stored in directory: /root/.cache/pip/wheels/31/d2/ed/fb9a0d301dd9586c11e9547120278e624227f22fd5f4baf744
Successfully built tflearn
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
from google.colab import files
files.upload()

with open('intents.json') as json_data:
    intents = json.load(json_data)

intents

Saving intents.json to intents.json


{'intents': [{'context_set': '',
   'patterns': ['What is a coronavirus', 'Tell me about coronavirus'],
   'responses': ['Coronaviruses are a large family of viruses which may cause illness in animals or humans.  In humans, several coronaviruses are known to cause respiratory infections ranging from the common cold to more severe diseases such as Middle East Respiratory Syndrome (MERS) and Severe Acute Respiratory Syndrome (SARS). The most recently discovered coronavirus causes coronavirus disease COVID-19.'],
   'tag': 'coronavirus'},
  {'context_set': '',
   'patterns': ['cya',
    'See you later',
    'Goodbye',
    'I am Leaving',
    'Have a Good day',
    'leaving'],
   'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'],
   'tag': 'goodbye'},
  {'context_set': '',
   'patterns': ['Hi',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day',
    'Whats up',
    "It's nice to meet you"],
   'responses': ['Hello!',
    'Good to see you again!',
  

In [3]:
words = []
classes = []
documents = []
ignore = ['?']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

99 documents
22 classes ['HandwashingTips', 'How_many_cases', 'NegativeTestResults', 'News_Corona_Virus', 'PrepareFamily', 'RiskToChildren', 'Testing', 'Wearingfacemasks', 'Whythename', 'antibiotics', 'catch', 'coronavirus', 'covid19', 'goodbye', 'greeting', 'medicine', 'protection', 'spread', 'spreadviafood', 'symptoms', 'vaccine', 'worry']
171 unique stemmed words ["'s", ',', '19', '2019', 'a', 'abbrevy', 'about', 'against', 'ahead', 'am', 'an', 'and', 'antibiot', 'any', 'anyon', 'anyth', 'ar', 'as', 'at', 'be', 'becom', 'been', 'being', 'by', 'cal', 'can', 'cas', 'catch', 'caus', 'cdc', 'child', 'childr', 'commun', 'coron', 'coronavir', 'correct', 'covid', 'covid-19', 'cur', 'cya', 'day', 'detect', 'diseas', 'do', 'doct', 'doe', 'drug', 'effect', 'emerg', 'everyon', 'fac', 'facemask', 'famy', 'far', 'feel', 'food', 'for', 'frequ', 'froz', 'get', 'goe', 'good', 'goodby', 'grocery', 'hand', 'handwash', 'hap', 'has', 'hav', 'head', 'hello', 'help', 'hi', 'high', 'hit', 'hom', 'househol

In [4]:
training = []
output = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [5]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 12999  | total loss: 0.00107 | time: 0.041s
| Adam | epoch: 1000 | loss: 0.00107 - acc: 1.0000 -- iter: 96/99
Training Step: 13000  | total loss: 0.00114 | time: 0.044s
| Adam | epoch: 1000 | loss: 0.00114 - acc: 1.0000 -- iter: 99/99
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [6]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

with open('intents.json') as json_data:
    intents = json.load(json_data)

model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [7]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [8]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    results = model.predict([bow(sentence, words)])[0]
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list

In [9]:
def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    if results:
        while results:
            for i in intents['intents']:
                if i['tag'] == results[0][0]:
                    return print("CovBot :",random.choice(i['responses']))

            results.pop(0)


In [11]:
flag=True
print("Hey, my name is CovBot, welcome to Chatbot. I will answer your queries. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("CovBot : You are welcome..")
        else:
            if(response(user_response)!=None):
                print("CovBot :",response(user_response))
    else:
        flag=False
        print("CovBot : Bye! take care..")

Hey, my name is CovBot, welcome to Chatbot. I will answer your queries. If you want to exit, type Bye!
covid 19 symptoms
CovBot : The most common symptoms of COVID-19 are fever, tiredness, and dry cough. Some patients may have aches and pains, nasal congestion, runny nose, sore throat or diarrhea. These symptoms are usually mild and begin gradually. Some people become infected but donâ€™t develop any symptoms and don't feel unwell. Most people (about 80%) recover from the disease without needing special treatment. Around 1 out of every 6 people who gets COVID-19 becomes seriously ill and develops difficulty breathing. Older people, and those with underlying medical problems like high blood pressure, heart problems or diabetes, are more likely to develop serious illness. People with fever, cough and difficulty breathing should seek medical attention.
How are you
CovBot : Hello!
Tell me about covid 19
CovBot : COVID-19 is the infectious disease caused by the most recently discovered coro